# Bysykkel i Trondheim

Du skal være konsulent for en dag, og din første kunde er Trondheim Bysykkel (et samarbeid mellom Trondheim Kommune og Clear Channel Norway AS).

Trondheim Kommune ønsker å tilby elsykler på de strekningene hvor folk bruker mest tid fra A til B, men de vet ikke hvilke strekgninger det gjelder. De ønsker denne informasjonen i både tabulær format og visualisert i et kart. 

De har gitt deg et Excel ark med ~140,000 turer fra 2021, og regner med at du klarer det på noen dager. Heldigvis kan du litt Pandas, og vet at du får til dette på under en time.

## Last inn data

In [ ]:
# Colab dependencies. Uncomment, run cell, and restart runtime before continuing.

# !pip install pandas-profiling==3.1.0 -q
# !pip install pydeck -q

In [1]:
# Import packages

import pandas as pd

In [2]:
# Load data

df = pd.read_excel('https://github.com/rmd-davis/bysykkel/blob/main/data/bysykkel_2021.xlsx?raw=true')

In [3]:
# Inspect data

df.head(10)

,Kurs i Pandas med NoA Ignite,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,Dato:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,started_at,ended_at,start_station_id,start_station_description,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_description,end_station_latitude,end_station_longitude
6,2021-04-07 04:21:06.812000+00:00,2021-04-07 04:33:22.530000+00:00,118,Ved Udbyes gate,63.416143,10.396315,108,Bassengbakken,ved Rosenborgbassenget,63.43592,10.414788
7,2021-04-07 05:56:15.319000+00:00,2021-04-07 06:00:36.670000+00:00,66,Ved Brattørbrua,63.435277,10.405814,107,Pirbadet,Ved Havnegata,63.440758,10.40217
8,2021-04-07 06:28:56.205000+00:00,2021-04-07 06:34:11.292000+00:00,51,ved kryss Mellomila / Illevollen,63.430403,10.37477,126,Leutenhaven,i Repslagerveita mot Prinsens gate. Operativt ...,63.429974,10.391444
9,2021-04-07 06:35:53.300000+00:00,2021-04-07 06:48:43.457000+00:00,108,ved Rosenborgbassenget,63.43592,10.414788,123,Hesthagen,Ved Klæbuveien,63.415418,10.399565


Her må vi rydde litt...
I Excel arket ligger kolonnenavnene i rad #7, så vi må bruke 'header' parameteren. Husk at Python er null-indeksert.

https://pandas.pydata.org/docs/reference/api/pandas.read_excel.html

In [4]:
# Load data with header parameter

df = pd.read_excel('https://github.com/rmd-davis/bysykkel/blob/main/data/bysykkel_2021.xlsx?raw=true', header=6)

In [5]:
# Inspect data after adding header parameter. 
# The first row whould contain a trip from Station 118 to Station 108

df.head()

,started_at,ended_at,start_station_id,start_station_description,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_description,end_station_latitude,end_station_longitude
0,2021-04-07 04:21:06.812000+00:00,2021-04-07 04:33:22.530000+00:00,118,Ved Udbyes gate,63.416143,10.396315,108,Bassengbakken,ved Rosenborgbassenget,63.435920,10.414788
1,2021-04-07 05:56:15.319000+00:00,2021-04-07 06:00:36.670000+00:00,66,Ved Brattørbrua,63.435277,10.405814,107,Pirbadet,Ved Havnegata,63.440758,10.402170
2,2021-04-07 06:28:56.205000+00:00,2021-04-07 06:34:11.292000+00:00,51,ved kryss Mellomila / Illevollen,63.430403,10.374770,126,Leutenhaven,i Repslagerveita mot Prinsens gate. Operativt ...,63.429974,10.391444
3,2021-04-07 06:35:53.300000+00:00,2021-04-07 06:48:43.457000+00:00,108,ved Rosenborgbassenget,63.435920,10.414788,123,Hesthagen,Ved Klæbuveien,63.415418,10.399565
4,2021-04-07 06:38:53.048000+00:00,2021-04-07 06:43:53.648000+00:00,51,ved kryss Mellomila / Illevollen,63.430403,10.374770,28,Kongens gate,Ved Nordre gate,63.430457,10.398101


## Automatic EDA (Exploratory Data Analysis)

Nå kan vi bruke Pandas Profiling for å få et raskt innblikk i datasettet vårt.

https://pypi.org/project/pandas-profiling/

In [6]:
from pandas_profiling import ProfileReport

profile = ProfileReport(df, title='Bysykkel Trondheim', html={'style':{'full_width':True}})
profile.to_notebook_iframe()

Render HTML: 100%|██████████| 1/1 [00:01<00:00,  1.11s/it]


## Data cleanup

Problemer med datasettet:

* 'started_at' og 'ended_at' er strings og ikke datetime variabler
* Mangler 'start_station_name'
* Har noen ekstra kolonner vi ikke trenger
* Duplicate rows
* Mangler 'duration' for hver tur 
* 'ended_at' mangler i 703 tilfeller

### Datoer

'started_at' og 'ended_at' er 'objects', men skal være 'datetime'
<br>https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html

In [7]:
# Use df.info() to get basic information about the dataset. Notice the Dtype for 'started_at' and 'ended_at'.

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137160 entries, 0 to 137159
Data columns (total 11 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   started_at                 137160 non-null  object 
 1   ended_at                   136457 non-null  object 
 2   start_station_id           137160 non-null  int64  
 3   start_station_description  136297 non-null  object 
 4   start_station_latitude     137160 non-null  float64
 5   start_station_longitude    137160 non-null  float64
 6   end_station_id             137160 non-null  int64  
 7   end_station_name           137160 non-null  object 
 8   end_station_description    136917 non-null  object 
 9   end_station_latitude       137160 non-null  float64
 10  end_station_longitude      137160 non-null  float64
dtypes: float64(4), int64(2), object(5)
memory usage: 11.5+ MB


In [8]:
# Convert 'started_at' and 'ended_at' to datetime variables

df['started_at'] = pd.to_datetime(df['started_at'])
df['ended_at'] = pd.to_datetime(df['ended_at'])

In [9]:
# Use df.info() again, and now notice the Dtype for 'started_at' and 'ended_at'

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137160 entries, 0 to 137159
Data columns (total 11 columns):
 #   Column                     Non-Null Count   Dtype              
---  ------                     --------------   -----              
 0   started_at                 137160 non-null  datetime64[ns, UTC]
 1   ended_at                   136457 non-null  datetime64[ns, UTC]
 2   start_station_id           137160 non-null  int64              
 3   start_station_description  136297 non-null  object             
 4   start_station_latitude     137160 non-null  float64            
 5   start_station_longitude    137160 non-null  float64            
 6   end_station_id             137160 non-null  int64              
 7   end_station_name           137160 non-null  object             
 8   end_station_description    136917 non-null  object             
 9   end_station_latitude       137160 non-null  float64            
 10  end_station_longitude      137160 non-null  float64     

### Missing data
Legg til 'start_station_name' basert på 'start_station_id', fra Excel arket som ligger i <em>'data/start_station_id.xlsx'</em>.
<br>Her kan vi bruke df.merge( ) 
<br>https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html 

In [10]:
# Load station names Excel file into a dataframe

station_names = pd.read_excel('https://github.com/rmd-davis/bysykkel/blob/main/data/start_station_id.xlsx?raw=true')

In [11]:
# Inspect station_names dataframe
station_names

,start_station_id,start_station_name
0,118,Abels gate
1,66,Verftsbrua
2,51,Skansen
3,108,Bassengbakken
4,21,Bakke bru
...,...,...
64,2303,Tempe
65,2302,Holtermanns veg 1
66,128,Lillegårdsbakken
67,2324,SLUPPEN TEST


In [12]:
# Create a merged dataframe using the original df and station_names
merged_frame = df.merge(station_names, on='start_station_id', how='left')

# Inspect created merged_frame. Notice that 'station_names' is added to the last column.
merged_frame.head(1)



,started_at,ended_at,start_station_id,start_station_description,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_description,end_station_latitude,end_station_longitude,start_station_name
0,2021-04-07 04:21:06.812000+00:00,2021-04-07 04:33:22.530000+00:00,118,Ved Udbyes gate,63.416143,10.396315,108,Bassengbakken,ved Rosenborgbassenget,63.43592,10.414788,Abels gate


In [13]:
# Extract 'start_station_name' series as a column
station_names_column = merged_frame['start_station_name']

# Insert 'station_names_column' into original df in position 3 (after start_station_id)
df.insert(loc=3,column='start_station_name', value=station_names_column)

# Inspect resulting df
df.head()

,started_at,ended_at,start_station_id,start_station_name,start_station_description,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_description,end_station_latitude,end_station_longitude
0,2021-04-07 04:21:06.812000+00:00,2021-04-07 04:33:22.530000+00:00,118,Abels gate,Ved Udbyes gate,63.416143,10.396315,108,Bassengbakken,ved Rosenborgbassenget,63.435920,10.414788
1,2021-04-07 05:56:15.319000+00:00,2021-04-07 06:00:36.670000+00:00,66,Verftsbrua,Ved Brattørbrua,63.435277,10.405814,107,Pirbadet,Ved Havnegata,63.440758,10.402170
2,2021-04-07 06:28:56.205000+00:00,2021-04-07 06:34:11.292000+00:00,51,Skansen,ved kryss Mellomila / Illevollen,63.430403,10.374770,126,Leutenhaven,i Repslagerveita mot Prinsens gate. Operativt ...,63.429974,10.391444
3,2021-04-07 06:35:53.300000+00:00,2021-04-07 06:48:43.457000+00:00,108,Bassengbakken,ved Rosenborgbassenget,63.435920,10.414788,123,Hesthagen,Ved Klæbuveien,63.415418,10.399565
4,2021-04-07 06:38:53.048000+00:00,2021-04-07 06:43:53.648000+00:00,51,Skansen,ved kryss Mellomila / Illevollen,63.430403,10.374770,28,Kongens gate,Ved Nordre gate,63.430457,10.398101


### Slett unødvendige kolonner

Slett 'start_station_description' og 'end_station_description'. 
<br>Bruk df.drop( ).
<br> https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop.html


In [14]:
# Delete columns using df.drop()
df = df.drop(columns=['start_station_description','end_station_description'])

### Fjern duplicates

In [15]:
# Find duplicated rows
df[df.duplicated(keep=False)]


,started_at,ended_at,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude
29142,2021-05-24 06:45:44.267000+00:00,2021-05-24 07:01:19.249000+00:00,19,Jomfrugateallmenningen,63.434423,10.395468,236,Lade idrettsanlegg vest,63.445964,10.444293
29143,2021-05-24 06:45:44.267000+00:00,2021-05-24 07:01:19.249000+00:00,19,Jomfrugateallmenningen,63.434423,10.395468,236,Lade idrettsanlegg vest,63.445964,10.444293


In [16]:
# Inspect duplicate row using df.loc
df.loc[29143]

started_at                 2021-05-24 06:45:44.267000+00:00
ended_at                   2021-05-24 07:01:19.249000+00:00
start_station_id                                         19
start_station_name                   Jomfrugateallmenningen
start_station_latitude                            63.434423
start_station_longitude                           10.395468
end_station_id                                          236
end_station_name                    Lade idrettsanlegg vest
end_station_latitude                              63.445964
end_station_longitude                             10.444293
Name: 29143, dtype: object

In [17]:
# Drop duplicate row

df = df.drop(29143, axis=0)

### Legg til en kolonne med tid
Legg til en kolonne som inneholder hvor lang tid hver tur tok i sekunder.
<br>Siden vi har 'started_at' og 'ended_at' klarer Pandas å regne ut tidsforskjellen automatisk som en timedelta.
<br>Bruk <code>df[column].dt.seconds</code> for å få sekunder som en float.

In [18]:
%%time

# Calculate timedelta between 'ended_at' and 'started_at'
df['duration'] = df['ended_at'] - df['started_at']

# Convert column from timedelta to float of seconds
df['duration'] = df['duration'].dt.seconds

Wall time: 15.8 ms


In [19]:
%%time

df['duration_lambda'] = df.apply(lambda x: (x['ended_at'] - x['started_at']).seconds, axis=1)


Wall time: 2.77 s


In [20]:
%%time

duration_list = []
duration_value = 0

for row in df.index:
    duration_value = (df.loc[row]['ended_at'] - df.loc[row]['started_at']).seconds
    duration_list.append(duration_value)

df['duration_loop'] = duration_list


Wall time: 25 s


### Fiks manglende verdier i 'ended_at'
* Finn gjennomsnittet av 'duration' for hvert segment i datasettet
* Bruk det som en lookup for hver rad som mangler 'duration'
* Bruk 'duration' + 'started_at' for å regne ut 'ended_at'

Se hvor mange unike segmenter det er

In [21]:
# Make unique segment ids based on start station and end station
df['segment'] = df['start_station_id'].astype(str) + '_' + df['end_station_id'].astype(str)

unique_segments = df['segment'].unique()

print(f'Total number of unique segments: {len(unique_segments)}')


Total number of unique segments: 4311


In [22]:
# Create dictionary of segment names and mean duration for each segment
segment_means = {}

# Loop over all of the unique segments and calculate the mean duration for that segment based on all trips for that segment

for segment in unique_segments:
    temp_frame = df[df['segment'] == segment]
    segment_means[segment] = temp_frame['duration'].mean()
    

In [23]:
# Extract dataframe with missing ended_at values
# Use df.isna()

missing_endtime = df[df['ended_at'].isna()]
missing_endtime = missing_endtime[['started_at','segment']]

# Show missing_endtime dataframe
missing_endtime

,started_at,segment
5,2021-04-07 06:38:58.614000+00:00,21_30
72,2021-04-07 16:39:30.615000+00:00,29_28
106,2021-04-08 11:13:07.769000+00:00,6_21
140,2021-04-08 15:03:46.120000+00:00,41_43
174,2021-04-08 17:40:18.258000+00:00,19_126
...,...,...
28904,2021-05-23 14:43:52.752000+00:00,41_18
28972,2021-05-23 17:03:24.613000+00:00,30_41
29006,2021-05-23 18:02:50.399000+00:00,95_51
29074,2021-05-23 21:27:38.510000+00:00,30_30


In [24]:
# Fill in missing durations with values from segment_means dictionary

missing_endtime['duration'] = missing_endtime['segment'].apply(lambda x: segment_means[x]) 

# Create timedelta, needed for calculation of ended_at
missing_endtime['duration_dt'] = pd.to_timedelta(missing_endtime['duration'], unit='s')

# Calculate ended_at
missing_endtime['ended_at'] = missing_endtime['started_at'] + missing_endtime['duration_dt']

# Extract necessary columns, needed to fill original dataframe
missing_endtime = missing_endtime[['ended_at','duration']]

# Show updated dataframe
missing_endtime

,ended_at,duration
5,2021-04-07 06:49:15.547333333+00:00,616.933333
72,2021-04-07 16:45:15.085085470+00:00,344.470085
106,2021-04-08 11:19:37.582953488+00:00,389.813953
140,2021-04-08 15:07:54.420613497+00:00,248.300613
174,2021-04-08 17:47:40.391333333+00:00,442.133333
...,...,...
28904,2021-05-23 14:53:40.580125+00:00,587.828125
28972,2021-05-23 17:11:02.411611111+00:00,457.798611
29006,2021-05-23 18:18:31.126272727+00:00,940.727273
29074,2021-05-23 21:49:53.103220339+00:00,1334.593220


In [25]:
# Fill missing values in original dataframe based on values in missing_endtime dataframe
# df.fillna() will automatically match based on index to fill in 'ended_at' and 'duration' values

df = df.fillna(missing_endtime)

## Sjekk resultatet av data cleanup

In [26]:
# Use df.describe() or df.describe().transpose()
df.describe()

,start_station_id,start_station_latitude,start_station_longitude,end_station_id,end_station_latitude,end_station_longitude,duration,duration_lambda,duration_loop
count,137159.000000,137159.000000,137159.000000,137159.000000,137159.000000,137159.000000,137159.000000,136456.000000,136456.000000
mean,298.681807,63.428478,10.403716,292.699976,63.429018,10.402294,688.329882,688.368720,688.368720
std,618.308959,0.008748,0.019730,615.853974,0.008580,0.018532,808.814671,810.508887,810.508887
min,1.000000,63.398407,10.357376,1.000000,63.398407,10.357376,61.000000,61.000000,61.000000
25%,41.000000,63.421867,10.393931,41.000000,63.423088,10.393931,340.000000,339.000000,339.000000
50%,76.000000,63.430403,10.399565,65.000000,63.430457,10.399565,502.000000,501.000000,501.000000
75%,123.000000,63.434423,10.412773,123.000000,63.434839,10.411413,754.000000,753.000000,753.000000
max,2325.000000,63.445964,10.459615,2325.000000,63.445964,10.459615,25726.000000,25726.000000,25726.000000


In [33]:
# Import packages for plotting

from matplotlib import pyplot as plt 
import seaborn as sns

In [39]:
# Plot duration histogram

fig,ax = plt.subplots(figsize = (15,10))
fig = sns.histplot(data = df, x = 'duration')

## Plot de 20 lengste turene

In [28]:
import pydeck as pdk

In [29]:
df_map_data = df.sort_values(by='duration', na_position='first').tail(20)
df_map_data

,started_at,ended_at,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,duration,duration_lambda,duration_loop,segment
137159,2021-11-30 16:37:48.976000+00:00,2021-11-30 23:46:16.535000+00:00,89,Tyholttårnet,63.422255,10.427769,293,S. P. Andersens vei,63.409888,10.405213,25707.0,25707.0,25707.0,89_293
137158,2021-11-30 16:37:47.976000+00:00,2021-11-30 23:46:16.535000+00:00,29,Singsaker,63.423121,10.412453,89,Tyholttårnet,63.422255,10.427769,25708.0,25708.0,25708.0,29_89
137157,2021-11-30 16:37:46.976000+00:00,2021-11-30 23:46:16.535000+00:00,133,Høyskoleringen,63.416901,10.406886,29,Singsaker,63.423121,10.412453,25709.0,25709.0,25709.0,133_29
137156,2021-11-30 16:37:45.976000+00:00,2021-11-30 23:46:16.535000+00:00,1878,Idrettsbygget Gløshaugen,63.420782,10.404758,133,Høyskoleringen,63.416901,10.406886,25710.0,25710.0,25710.0,1878_133
137155,2021-11-30 16:37:44.976000+00:00,2021-11-30 23:46:16.535000+00:00,41,Vollabakken,63.421867,10.399269,1878,Idrettsbygget Gløshaugen,63.420782,10.404758,25711.0,25711.0,25711.0,41_1878
137154,2021-11-30 16:37:43.976000+00:00,2021-11-30 23:46:16.535000+00:00,45,Studentersamfundet,63.421859,10.394516,41,Vollabakken,63.421867,10.399269,25712.0,25712.0,25712.0,45_41
137153,2021-11-30 16:37:42.976000+00:00,2021-11-30 23:46:16.535000+00:00,55,Finalebanen,63.419785,10.392211,45,Studentersamfundet,63.421859,10.394516,25713.0,25713.0,25713.0,55_45
137152,2021-11-30 16:37:41.976000+00:00,2021-11-30 23:46:16.535000+00:00,2302,Holtermanns veg 1,63.414747,10.397386,55,Finalebanen,63.419785,10.392211,25714.0,25714.0,25714.0,2302_55
137151,2021-11-30 16:37:40.976000+00:00,2021-11-30 23:46:16.535000+00:00,293,S. P. Andersens vei,63.409888,10.405213,2302,Holtermanns veg 1,63.414747,10.397386,25715.0,25715.0,25715.0,293_2302
137150,2021-11-30 16:37:39.976000+00:00,2021-11-30 23:46:16.535000+00:00,25,Thornesparken,63.432900,10.412773,21,Bakke bru,63.432252,10.406996,25716.0,25716.0,25716.0,25_21


In [30]:
df_map_data['start_coord'] = df_map_data.apply(lambda x: [x['start_station_longitude'],x['start_station_latitude']],axis=1)
df_map_data['end_coord'] = df_map_data.apply(lambda x: [x['end_station_longitude'],x['end_station_latitude']], axis=1)

In [31]:
layer = pdk.Layer("LineLayer", df_map_data, get_source_position="start_coord",get_target_position="end_coord", get_color = [255,0,0], get_width = 3)
init_view_state = pdk.ViewState(longitude=10.3985, latitude=63.4256, zoom=12)
r = pdk.Deck(layers=layer, initial_view_state=init_view_state, map_style='light')
r

{"initialViewState": {"latitude": 63.4256, "longitude": 10.3985, "zoom": 12}, "layers": [{"@@type": "LineLayer", "data": [{"duration": 25707.0, "duration_lambda": 25707.0, "duration_loop": 25707.0, "end_coord": [10.4052129025639, 63.40988844886488], "end_station_id": 293, "end_station_latitude": 63.40988844886488, "end_station_longitude": 10.4052129025639, "end_station_name": "S. P. Andersens vei", "ended_at": {}, "segment": "89_293", "start_coord": [10.42776882648468, 63.42225473306107], "start_station_id": 89, "start_station_latitude": 63.42225473306107, "start_station_longitude": 10.42776882648468, "start_station_name": "Tyholtt\u00e5rnet", "started_at": {}}, {"duration": 25708.0, "duration_lambda": 25708.0, "duration_loop": 25708.0, "end_coord": [10.42776882648468, 63.42225473306107], "end_station_id": 89, "end_station_latitude": 63.42225473306107, "end_station_longitude": 10.42776882648468, "end_station_name": "Tyholtt\u00e5rnet", "ended_at": {}, "segment": "29_89", "start_coord": [10.41245341300964, 63.42312115738324], "start_station_id": 29, "start_station_latitude": 63.42312115738324, "start_station_longitude": 10.41245341300964, "start_station_name": "Singsaker", "started_at": {}}, {"duration": 25709.0, "duration_lambda": 25709.0, "duration_loop": 25709.0, "end_coord": [10.41245341300964, 63.42312115738324], "end_station_id": 29, "end_station_latitude": 63.42312115738324, "end_station_longitude": 10.41245341300964, "end_station_name": "Singsaker", "ended_at": {}, "segment": "133_29", "start_coord": [10.40688648819923, 63.41690080192888], "start_station_id": 133, "start_station_latitude": 63.41690080192888, "start_station_longitude": 10.40688648819923, "start_station_name": "H\u00f8yskoleringen", "started_at": {}}, {"duration": 25710.0, "duration_lambda": 25710.0, "duration_loop": 25710.0, "end_coord": [10.40688648819923, 63.41690080192888], "end_station_id": 133, "end_station_latitude": 63.41690080192888, "end_station_longitude": 10.40688648819923, "end_station_name": "H\u00f8yskoleringen", "ended_at": {}, "segment": "1878_133", "start_coord": [10.4047576414641, 63.42078238303412], "start_station_id": 1878, "start_station_latitude": 63.42078238303412, "start_station_longitude": 10.4047576414641, "start_station_name": "Idrettsbygget Gl\u00f8shaugen", "started_at": {}}, {"duration": 25711.0, "duration_lambda": 25711.0, "duration_loop": 25711.0, "end_coord": [10.4047576414641, 63.42078238303412], "end_station_id": 1878, "end_station_latitude": 63.42078238303412, "end_station_longitude": 10.4047576414641, "end_station_name": "Idrettsbygget Gl\u00f8shaugen", "ended_at": {}, "segment": "41_1878", "start_coord": [10.39926901459694, 63.42186711370097], "start_station_id": 41, "start_station_latitude": 63.42186711370097, "start_station_longitude": 10.39926901459694, "start_station_name": "Vollabakken ", "started_at": {}}, {"duration": 25712.0, "duration_lambda": 25712.0, "duration_loop": 25712.0, "end_coord": [10.39926901459694, 63.42186711370097], "end_station_id": 41, "end_station_latitude": 63.42186711370097, "end_station_longitude": 10.39926901459694, "end_station_name": "Vollabakken ", "ended_at": {}, "segment": "45_41", "start_coord": [10.39451614022255, 63.42185871322338], "start_station_id": 45, "start_station_latitude": 63.42185871322338, "start_station_longitude": 10.39451614022255, "start_station_name": "Studentersamfundet", "started_at": {}}, {"duration": 25713.0, "duration_lambda": 25713.0, "duration_loop": 25713.0, "end_coord": [10.39451614022255, 63.42185871322338], "end_station_id": 45, "end_station_latitude": 63.42185871322338, "end_station_longitude": 10.39451614022255, "end_station_name": "Studentersamfundet", "ended_at": {}, "segment": "55_45", "start_coord": [10.39221078157425, 63.41978492000792], "start_station_id": 55, "start_station_latitude": 63.41978492000792, "start_station_longitude": 10.39221078157425, "start_station_name": "Finalebanen", "started_at": {}}, {"duration": 25714.0, "duration_lambda": 